## Set random seed

In [1]:
import torch
torch.manual_seed(42)

## Preprocess

In [2]:
def preprocess(path, max_len = None, training = True):
    dataset = []
    with open(path, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.strip()
            if training:
                dataset.append(['<SOS>'] + [word for word in line[:max_len]] + ['<EOS>'])
            else:
                dataset.append([word for word in line])
    return dataset

In [3]:
max_len = 25
dataset = preprocess('./data/hw2.1_corpus.txt', max_len)
testDataset = preprocess('./data/hw2.1-2_sample_testing_data.txt', training=False)

In [4]:
class Word_dict():
    def __init__(self, dataset, max_len):
        self.word_dict = {}
        self.idx_dict = {}
        self.add("<PAD>")
        for i in range(1, max_len + 1):
            self.add(str(i))
        self.add("<UNK>")

        self.make_dict(dataset)
        
    def make_dict(self, dataset):
        for sent in dataset:
            for word in sent:
                self.add(word)
        
    def add(self, word):
        if word not in self.word_dict:
            self.idx_dict[self.get_len()] = word
            self.word_dict[word] = self.get_len()
            
            
    def word2idx(self, word):
        if word in self.word_dict:
            return self.word_dict[word]
        else:
            return self.word_dict["<UNK>"]
    
    def idx2word(self, idx):
        return self.idx_dict[idx]
    
    def get_len(self):
        return len(self.word_dict)

In [5]:
word_dict = Word_dict(dataset, max_len)
PAD_IDX = word_dict.word2idx("<PAD>")
import pickle

with open('dicitonary.pkl','wb') as f:
    pickle.dump(word_dict, f)

In [6]:
def data2index(sent):
    return [word_dict.word2idx(word) for word in sent]

def process_training_data(dataset):
    datas = []
    for idx in range(len(dataset)-1):
        processed = {}
        processed['x'] = data2index(dataset[idx])
        processed['y'] = data2index(dataset[idx+1])
        datas.append(processed)
    return datas

def process_testing_data(dataset):
    datas = []
    for data in dataset:
        datas.append(data2index(data))
    return datas

In [7]:
from sklearn.model_selection import train_test_split

data = process_training_data(dataset)
trainData, validData = train_test_split(data, test_size=0.2, random_state=42)

testData = process_testing_data(testDataset)

In [8]:
import random
random.seed(42)

In [9]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class SentenceDataset(Dataset):
    def __init__(self, data, pad_idx, ncontrols = 1, training = True):
        self.data = data
        self.pad_idx = pad_idx
        self.max_len = max_len
        self.training = training
        self.ncontrols = ncontrols
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
        
    def collate_fn(self, datas):
        if self.training:
            processed_data = []
            for data in datas:
                self.process_data(data)
                processed_data.append(data)
#                 print(data)
                
            processed_data.sort(key=lambda x : len(x['x']), reverse=True)
            batch_data = [data['x'] for data in processed_data]
            
            datas_len = [len(data) for data in batch_data]
            batch_data = self.make_padding(batch_data)
            
            batch_label = [data['y'] for data in processed_data]
            batch_label = self.make_padding(batch_label)
            batch_random_num = [data['random_num'] for data in processed_data]
                        
            return torch.LongTensor(batch_data), torch.LongTensor(batch_label), torch.LongTensor(datas_len), batch_random_num
        
        else:
            datas_idx = sorted(range(len(datas)), key=lambda x : datas[x])
            datas.sort(key=lambda x: len(x), reverse=True)
            datas_len = [len(sent) for sent in datas]
            batch_data = self.make_padding(datas)
            return torch.LongTensor(batch_data), torch.LongTensor(datas_len), datas_idx
    
    def process_data(self, data):
        data['random_num'] = self.get_ncontrols(len(data['y'])-2)
        data['x'] = self.add_control(data['x'], data['y'], data['random_num'])
    
    def get_ncontrols(self, sent_len):
        sample_num = random.randint(1, self.ncontrols)
        if sent_len < sample_num:
            sample_num = sent_len
            
        samples = random.sample([i for i in range(1, sent_len+1)], sample_num)
        samples.sort()
        return samples
    
    def add_control(self, data, label, samples):
        for sample in samples:
            data = data + [word_dict.word2idx(str(sample))] + [label[sample]]
        return data
    
    def make_padding(self, datas):
        max_len = max([len(data) for data in datas])
        return [data + [self.pad_idx]*(max_len-len(data)) for data in datas]

In [10]:
trainData = SentenceDataset(trainData, PAD_IDX, 2)
validData = SentenceDataset(validData, PAD_IDX, 2)
testData = SentenceDataset(testData, PAD_IDX, training = False)

## Model

In [11]:
from torch import nn

In [12]:
class Embedding(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super(Embedding, self).__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
    def forward(self, input):
        embedded = self.embedding(input)
        
        return embedded

In [13]:
from torch.nn.utils.rnn import pack_padded_sequence
class Encoder(nn.Module):
    def __init__(self, emb_dim, hid_dim, embedding):
        super(Encoder, self).__init__()
        
        self.embedding = embedding
    
        self.rnn = nn.GRU(emb_dim, hid_dim, batch_first=True)        
    
    def forward(self, input, data_len):
        input = self.embedding(input)
        
        pack_datas = pack_padded_sequence(input, data_len, batch_first=True)
        
        outputs, hidden = self.rnn(input)
        
        return hidden

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, embedding):
        super(Decoder, self).__init__()
        
        self.output_dim = output_dim
        
        self.embedding = embedding
        
        self.rnn = nn.GRU(emb_dim, hid_dim, batch_first=True)

        self.out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, input, hidden):
        input = self.embedding(input)
        
        output, hidden = self.rnn(input, hidden)
        
        prediction = self.out(output)
        
        return prediction, hidden

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, src, trg = None, data_len = None, teacher_forcing_ratio = 0.5, training = True):
        batch_size = src.shape[0]
        if training:
            max_len = trg.shape[1]
        else:
            max_len = 25
        
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(batch_size, max_len-1, trg_vocab_size).to(device)
        
        hidden = self.encoder(src, data_len)
        
        input = src[:, 0:1]
        for t in range(1, max_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:,t-1:t] = output
            
            top1 = output.argmax(2)
            
            if torch.rand(1) > teacher_forcing_ratio:
                input = top1
            else:
                input = trg[:,t:t+1]
        return outputs

## Train

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
INPUT_DIM = word_dict.get_len()
OUTPUT_DIM = word_dict.get_len()
EMB_DIM = 128
HID_DIM = 128

emb = Embedding(INPUT_DIM, EMB_DIM)
enc = Encoder(EMB_DIM, HID_DIM, emb)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, emb)

model = Seq2Seq(enc, dec).to(device)

In [18]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(
      (embedding): Embedding(6583, 128)
    )
    (rnn): GRU(128, 128, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(
      (embedding): Embedding(6583, 128)
    )
    (rnn): GRU(128, 128, batch_first=True)
    (out): Linear(in_features=128, out_features=6583, bias=True)
  )
)


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,889,975 trainable parameters


In [20]:
class accuracy():
    def __init__(self):
        self.ncontrol_word = 0
        self.nword_correct = 0
    def update(self, predictions, groundTruths, random_nums):
        predictions = predictions.cpu()
        groundTruths = groundTruths.cpu()
        for i in range(len(groundTruths)):
            length = int(torch.sum(groundTruths[i] != PAD_IDX))
            prediction = predictions[i][:length]
            groundTruth = groundTruths[i][:length]
            random_num = random_nums[i]
            self.ncontrol_word += len(random_num)
            for num in random_num:
                if prediction[num-1] == groundTruth[num-1]:
                    self.nword_correct += 1
            
    
    def get_word_score(self):
        return self.nword_correct / self.ncontrol_word
    
    def print_word_score(self):
        score = self.get_word_score()
        return '{:.5f}'.format(score)

In [21]:
import os
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook as tqdm
import json

def run_epoch(epoch, training):
    model.train(training)
    if training:
        description = 'Train'
        dataset = trainData
        shuffle = True
    else:
        description = 'Valid'
        dataset = validData
        shuffle = False
    
    dataloader = DataLoader(dataset = dataset,
                            batch_size = batch_size,
                            shuffle=shuffle,
                            collate_fn=dataset.collate_fn,
                            num_workers=0)
    
    trange = tqdm(enumerate(dataloader), total=len(dataloader), desc=description)
    loss = 0
    acc = accuracy()
    for i, (x, y, data_len, random_num) in trange:
        output, sent_loss = run_iter(x, y, data_len, random_num, training)
        batch_loss = sent_loss
        if training:
            opt.zero_grad()
            batch_loss.backward()
            opt.step()
        loss += batch_loss.item()
        acc.update(output, y[:, 1:], random_num)
        trange.set_postfix(loss = loss / (i + 1), acc = acc.print_word_score())
    if training:
        history['train'].append({'acc':acc.get_word_score(), 'loss':loss/ len(trange)})
    else:
        history['valid'].append({'acc':acc.get_word_score(), 'loss':loss/ len(trange)})
def run_iter(x, y, data_len, random_nums, training):
    x = x.to(device)
    y = y.to(device)
    if training:
        output = model(x, y, data_len)
    else:
        with torch.no_grad():
            output = model(x, y, data_len, 0)
    sent = y[:, 1:]
    sent = sent.reshape(sent.shape[0] * sent.shape[1])
    output_sent = output.view(output.shape[0] * output.shape[1], output.shape[2])
    sent_loss = criterion(output_sent, sent)
        
    return output.argmax(2), sent_loss

def save(epoch):
    if not os.path.exists('model'):
        os.makedirs('model')
    torch.save(model.state_dict(), 'model/model.pkl.'+str(epoch))
    with open('model/history.json', 'w') as f:
        json.dump(history, f, indent=4)

In [22]:
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size = 10, gamma = 0.5)
max_epoch = 50
batch_size = 32
history = {'train':[],'valid':[]}

In [23]:
for epoch in range(1):
    print('Epoch: {}'.format(epoch))
    run_epoch(epoch, True)
    run_epoch(epoch, False)
    scheduler.step()
#     for p in model.parameters():
#         print(p)
#     save(epoch)

Epoch: 0


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

with open('model/history.json', 'r') as f:
    history = json.loads(f.read())
    
train_loss = [l['loss'] for l in history['train']]
valid_loss = [l['loss'] for l in history['valid']]
train_f1 = [l['acc'] for l in history['train']]
valid_f1 = [l['acc'] for l in history['valid']]

plt.figure(figsize=(7,5))
plt.title('Loss')
plt.plot(train_loss, label='train')
plt.plot(valid_loss, label='valid')
plt.legend()
plt.show()

plt.figure(figsize=(7,5))
plt.title('F1 Score')
plt.plot(train_f1, label='train')
plt.plot(valid_f1, label='valid')
plt.legend()
plt.show()

print('Best F1 score ', max([[l['acc'], idx] for idx, l in enumerate(history['valid'])]))

In [ ]:
model.load_state_dict(torch.load('model/model.pkl.{}'.format(1)))
model.train(False)
dataloader = DataLoader(dataset=testData,
                            batch_size=32,
                            shuffle=False,
                            collate_fn=testData.collate_fn,
                            num_workers=4)
trange = tqdm(enumerate(dataloader), total=len(dataloader), desc='Predict')
prediction = []
for i, (x, data_len, datas_idx) in trange:
    o_labels = model(x.to(device), data_len = data_len,teacher_forcing_ratio = 0.0, training = False)
    o_labels = torch.argmax(o_labels, dim = 2)
    o_labels = o_labels.to('cpu').numpy().tolist()
    for idx in datas_idx:
        prediction.append(o_labels[idx])


In [ ]:
with open('task2-2.txt', 'w', encoding="utf-8") as f:
    for idx, sent in enumerate(prediction):
        s = ' '
        seq = ["<SOS>"] + [word_dict.idx2word(word_idx) for word_idx in sent]
        f.writelines(s.join(seq) + '\n')